In [1]:
pip install pytesseract python-docx

Defaulting to user installation because normal site-packages is not writeable

   ---------------------------------------- 0/2 [python-docx]
   ---------------------------------------- 0/2 [python-docx]
   ---------------------------------------- 0/2 [python-docx]
   ---------------------------------------- 0/2 [python-docx]
   ---------------------------------------- 0/2 [python-docx]
   ---------------------------------------- 0/2 [python-docx]
   ---------------------------------------- 0/2 [python-docx]
   ---------------------------------------- 0/2 [python-docx]
   ---------------------------------------- 0/2 [python-docx]
   ---------------------------------------- 0/2 [python-docx]
   ---------------------------------------- 0/2 [python-docx]
   ---------------------------------------- 0/2 [python-docx]
   ---------------------------------------- 0/2 [python-docx]
   ---------------------------------------- 0/2 [python-docx]
   -------------------- ------------------- 1/2 [pyte

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pytesseract
from PIL import Image

# Tell Python where Tesseract is installed
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"

# Load the image
image = Image.open("sample.jpg")

# Extract text
text = pytesseract.image_to_string(image)

print("=== OCR Output ===")
print(text)

=== OCR Output ===
Pay V5C-0823
fh yhicle details

| Official use only

Atl ly

ier” __-GDO5 EGF

— B:Date of fret regstration 25 11 2015

{8.1}: Date of first
registration in the Ux 2> 11-2015

D.1: Make FORD.
D2: Type JAB
Variant SFJCAD
Version EUROB W5DHA BL
Euro status
Real driving emissions:
D.3: Model FIESTA ZETEC TURBO
DS: Body type 5 DOOR HATCHBACK
[A Taxation class PETROL CAR
(D.6}: Suspension type
|: Revenue weight 1555 KG GROSS
PA; Oylnder capacity (@<) 998 CC
fees) «
___ N:CO. (ar) 099 G/KM

‘daclanng that the information provided is correct.

q uf
1 Change my vehicle details — only fill in details to be corrected or changed

Document reference number

4102 482 2248
Wheelpian or Body type:

Cylinder capacity (cc):

VIN, Chassis or Frame number:

New revenue weight: Date of change:

2 Selling or transferring my vehicle to a new keeper (nota trader)

By submitting this form you are declaring that the Information provided |s correct,
You must tell us immediately if you have so

In [3]:
make = "FORD"
model = "FIESTA ZETEC TURBO"
year = "2015"
chasis = "WFODXXGAKDFT40114"
mileage = "42000"
reg_number = "GD65 EGF"

In [30]:
from docx import Document
import pytesseract
from PIL import Image
import re
import os

# Configuration
TESSERACT_PATH = r"C:\Program Files\Tesseract-OCR\tesseract.exe"
TEMPLATE_PATH = "sales_receipt.docx"
OUTPUT_PATH = "filled_receipt.docx"

def safe_extract(text, pattern, group=1):
    """Safely extract field with error handling"""
    try:
        match = re.search(pattern, text, re.IGNORECASE)
        return match.group(group).strip() if match else None
    except Exception:
        return None

def extract_reg_number(text):
    """Specialized function to extract registration number"""
    # Try to find GD65 EGF at the end of document
    last_lines = text.splitlines()[-10:]  # Check last 10 lines
    for line in last_lines:
        clean_line = re.sub(r"[^A-Z0-9]", "", line.upper())
        if re.fullmatch(r"GD65EGF", clean_line):
            return "GD65EGF"
    
    # Try near "Registration Number" text
    chunks = text.split("Registration Number")
    if len(chunks) > 1:
        next_lines = chunks[1].splitlines()[:5]  # Check next 5 lines
        for line in next_lines:
            clean_line = re.sub(r"[^A-Z0-9]", "", line.upper())
            if re.fullmatch(r"GD65EGF", clean_line):
                return "GD65EGF"
    
    return "N/A"

def extract_data_from_image(image_path):
    """Extracts all vehicle data from image"""
    pytesseract.pytesseract.tesseract_cmd = TESSERACT_PATH
    img = Image.open(image_path)
    text = pytesseract.image_to_string(img)
    
    # Print last 10 lines for debugging
    print("=== DOCUMENT END ===")
    print("\n".join(text.splitlines()[-10:]))
    
    data = {
        "make": safe_extract(text, r"D\.1: Make\s+([A-Z]+)") or "N/A",
        "model": safe_extract(text, r"D\.3: Model\s+([A-Za-z\s]+?)(?=\n|D\.5)") or "N/A",
        "year": safe_extract(text, r"(?:B:Date of|Date of first)[^\d]*(\d{2}\s+\d{2}\s+(\d{4}))", 2) or "N/A",
        "chasis": safe_extract(text, r"E: VIN/[A-Za-z]+/Frame No\s+([A-Z0-9]{17})") or "N/A",
        "mileage": safe_extract(text, r"Mileage\s*[:\(]?\s*(?:optional\s*\)?)?\s*:?\s*(\d[\d,]*)") or "N/A",
        "reg_number": extract_reg_number(text)
    }
    
    # Clean mileage if found
    if data["mileage"] != "N/A":
        data["mileage"] = data["mileage"].replace(",", "")
    
    return data

def fill_word_template(data, template_path, output_path):
    """Fills the Word template with extracted data"""
    doc = Document(template_path)
    
    replacements = {
        "{{make}}": data["make"],
        "{{model}}": data["model"],
        "{{year}}": data["year"],
        "{{chasis}}": data["chasis"],
        "{{mileage}}": data["mileage"],
        "{{reg_number}}": data["reg_number"]
    }

    for p in doc.paragraphs:
        for key, value in replacements.items():
            if key in p.text:
                p.text = p.text.replace(key, value)

    for table in doc.tables:
        for row in table.rows:
            for cell in row.cells:
                for key, value in replacements.items():
                    if key in cell.text:
                        cell.text = cell.text.replace(key, value)

    doc.save(output_path)

if __name__ == "__main__":
    if not all(os.path.exists(f) for f in [TEMPLATE_PATH, "sample.jpg"]):
        print("Error: Missing required files!")
    else:
        extracted_data = extract_data_from_image("sample.jpg")
        print("\n=== EXTRACTED DATA ===")
        print(f"MAKE        : {extracted_data['make']}")
        print(f"MODEL       : {extracted_data['model']}")
        print(f"YEAR        : {extracted_data['year']}")
        print(f"CHASIS      : {extracted_data['chasis']}")
        print(f"MILEAGE     : {extracted_data['mileage']}")
        print(f"REG_NUMBER  : {extracted_data['reg_number']}")
        
        fill_word_template(extracted_data, TEMPLATE_PATH, OUTPUT_PATH)
        print(f"\nReceipt saved to {OUTPUT_PATH}")

=== DOCUMENT END ===

Driving licence number of
the new keeper: (optional)

Il

GD65 EGF

Wt fo,


=== EXTRACTED DATA ===
MAKE        : FORD
MODEL       : FIESTA ZETEC TURBO
YEAR        : 2015
CHASIS      : WFODXXGAKDFT40414
MILEAGE     : N/A
REG_NUMBER  : GD65EGF

Receipt saved to filled_receipt.docx
